In [ ]:
# subcluster each lineage and annotate
import anndata
import scanpy as sc
import pandas as pd
import glob
import skimage
import re
import scanpy.external as sce
import matplotlib.pyplot as plt
from pathlib import Path

plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['pdf.fonttype'] = 42 #make text editable in pdf

import os
# Set working directory
lin='Mye'
os.chdir(f"/diskmnt/Projects/myeloma_scRNA_analysis/MMY_IRD/revision/merge/no_harmony/{lin}")

# Verify current working directory
print(os.getcwd())

# read in HGNC Ig locus genes
ig_genes_table = pd.read_csv(
    "/diskmnt/Projects/myeloma_scRNA_analysis/MMY_IRD/analysis/resources/IgGenes_HGNC_geneNamesOrg.txt",
    sep="\t",
    header=0
)
ig_genes = ig_genes_table['Approved symbol'].tolist()

# filter genes for looking at DEGs: only protein coding, no MT, IG, hemoglobins
protein_coding_genes_table = pd.read_csv("/diskmnt/Projects/myeloma_scRNA_analysis/MMY_IRD/analysis/resources/HGNC_protein_coding_gene.txt", sep="\t")
protein_coding_genes = protein_coding_genes_table['symbol'].tolist()

hemo_genes_table = pd.read_csv("/diskmnt/Projects/myeloma_scRNA_analysis/MMY_IRD/analysis/resources/hemoglobin_genes.txt", sep="\t")
hb_genes = hemo_genes_table['Gene'].tolist()

protein_coding_filtered = [
    g for g in protein_coding_genes
    if (
        g not in ig_genes and
        g not in hb_genes and
        not re.match(r"^(RPL|RPS|MT-)", g)
    )
]

In [ ]:
adata = sc.read_h5ad("split.h5ad")
adata

In [ ]:
# temporarily exclude Ig genes from variable gene selection
adata_hvg_calc = adata[:, ~adata.var_names.isin(ig_genes)].copy()

sc.pp.highly_variable_genes(
    adata_hvg_calc,
    n_top_genes=2000,
    batch_key='Sample'
)

# Initialize all genes in full object as False
adata.var['highly_variable'] = False

# Mark HVGs from the subset in the full object
adata.var.loc[adata_hvg_calc.var_names[adata_hvg_calc.var['highly_variable']], 'highly_variable'] = True

sc.tl.pca(adata)
sce.pp.harmony_integrate(adata, "Sample", max_iter_harmony=20)

adata.obsm['X_pca'] = adata.obsm['X_pca_harmony']
sc.pp.neighbors(adata)
sc.tl.umap(adata)

#determine resolution for clustering
for res in [0.1, 0.3, 0.5, 1.0]:
    sc.tl.leiden(
        adata, key_added=f"leiden_res_{res:4.2f}", resolution=res, flavor="igraph"
    )

In [ ]:
resolutions = ["leiden_res_0.10", "leiden_res_0.30", "leiden_res_0.50", "leiden_res_1.00" ]
fig, axes = plt.subplots(1,4, figsize=(16,5))
axes = axes.flatten()  # flatten to 1D for easy iteration

for ax, res in zip(axes, resolutions):
    sc.pl.umap(adata, color=res, ax=ax, legend_loc='on data', show=False)
    ax.set_aspect('equal') 
    for coll in ax.collections:
        coll.set_rasterized(True)

plt.tight_layout()
plt.savefig("leiden_umaps.pdf")

In [ ]:
m_markers = {
    #'MDSC': ['ARG1', 'NOS2', 'CD274', 'HIF1A', "CEBPB", "STAT3"],
    "HSPC": ["SPINK2", "AVP", "CD34"],
    "Granulo.":["CSF3R", 'MPO', 'ELANE', 'AZU1', "CEACAM8"],
    "Monocyte":['CSF1R', 'ITGAM', 'VCAN', 'THBS1', 'SPI1', 'IRF8'],
    "Classical Mono.": ['CD14', 'CCR2', 'FCGR1A'],
    "Inflammatory": ['IL1B', 'CCL2', 'IL6', 'CXCL8', 'S100A8', 'S100A9', 'LYZ', 'STAT1', 'NLRP3'],
    "Immunosupp.": ['STAT3', 'STAT6', 'TGFB1', 'KLF4'],
    "Non-class. Mono.": ['FCGR3A', 'CX3CR1', 'ITGAX', 'ITGAL', 'NR4A1', 'CDKN1C'],
    'TAM': ['CD163', 'CD68', 'MS4A7', 'MARCO', 'MSR1', 'SIGLEC10', 'C1QA', 'C1QB'],
    "Ag Presenting": ['HLA-DPB1', 'HLA-DQA1', 'HLA-DQB1', 'HLA-DRB1'],
    "cDC": ['CLEC9A', 'C1orf54', 'IRF8', 'CLEC10A', 'CD1C','FCER1A'],
    'MSC': ['LEPR', 'KITLG', 'CXCL12'],
    "T": ["TRAC", "CD3E", "CD3D", "NKG7", "GNLY", "KLRD1"],
    #'B': ['CD19', 'CD79A', 'MS4A1'],
    #'PC':['SDC1', 'MZB1', 'SLAMF7'],
    'MKC':['PF4', 'PPBP'],
    'Ery': ['HBB', 'HBD', 'AHSP']
    
}
sc.pl.dotplot(adata, m_markers, groupby="leiden_res_0.50", layer='counts', standard_scale="var")


In [ ]:
pd.crosstab(adata.obs["leiden_res_0.50"], adata.obs["Collection"])

In [ ]:
adata.obs["Collection"].value_counts()

In [ ]:
print(340/22835)
print(1324/82924)
print(2644/116376)
print(142/3481)

In [ ]:
sc.pl.umap(adata, color='nCount_RNA', size=3)

In [ ]:
cluster2celltype = {
    "0": "CD16 Mc / TAM", 
    "1": 'CD14 Mc',
    "2": "CD14 Mc", 
    "3": "Low_Quality", # low nCount mixed monocytes 
    "4": "CD14 Mc",
    "5": "Neutrophil", 
    "6": "Low_Quality", # Ery doublet 
    "7": "cDC",
    "8": "Low_Quality", # T doublet
    "9": "Low_Quality", # Ery doublet 
    "10": "HSPC",
    "11": "MSC"
}

adata.obs["subset"] = adata.obs["leiden_res_0.50"].map(cluster2celltype)
adata_cleaned = adata[adata.obs['subset']!='Low_Quality'].copy()

In [ ]:
subset_order  = [
    "HSPC",
    "Neutrophil",
    "CD14 Mc",
    "CD16 Mc / TAM", 
    "cDC", 
    "MSC"
]

markers_simple = {
    "HSPC": ["SPINK2", "AVP", "CD34"],
    "Granulo.":['MPO', 'ELANE', 'AZU1', 'LYZ', 'CSF3R'],
    "Classical Mc": ['CD14', 'CCR2', 'FCGR1A', 'IRF8', 'ITGAM' ],
    "Inflammatory": ['IL1B', 'CXCL8', 'S100A8', 'S100A9', 'STAT1', 'NLRP3'],
    "Immunosupp.": ['STAT3', 'STAT6', 'TGFB1'],
    "Non-class. Mc": ['FCGR3A', 'CX3CR1', 'ITGAX', 'ITGAL', 'NR4A1', 'CDKN1C'],
    'TAM': ['CSF1R', 'SPI1', 'CD163', 'CD68', 'MS4A7', 'MARCO', 'MSR1', 'SIGLEC10', 'C1QA', 'C1QB'],
    "MHC-II": ['HLA-DPB1', 'HLA-DQA1', 'HLA-DQB1', 'HLA-DRB1'],
    "cDC": ['CLEC9A', 'C1orf54', 'IRF8', 'CLEC10A', 'CD1C','FCER1A'],
    'MSC': ['CCL2', 'VCAN', 'THBS1', 'LEPR', 'KITLG', 'CXCL12'],
    
}

sc.pl.dotplot(
    adata_cleaned,
    var_names=markers_simple,
    layer='counts',
    groupby="subset",
    categories_order=subset_order,   # explicit order
    standard_scale="var",
    figsize=(16,2),
    show=False
)

plt.savefig("subset_markers_dotplot.pdf", bbox_inches="tight")

In [ ]:
lin_rename = {
    "HSPC": "HSPC",
    "Neutrophil": "Mye",
    "CD14 Mc": "Mye",
    "CD16 Mc / TAM": "Mye", 
    "cDC": "Mye", 
    "MSC": "MSC"
}

adata_cleaned.obs["lin"] = adata_cleaned.obs["subset"].map(lin_rename)

In [ ]:
adata_cleaned.obs["subset"].unique()

In [ ]:
# Assign colors to each subset
palette = {
    "HSPC": "#d6e376",
    "Neutrophil": "#95ad74",
    "CD14 Mc": "#64d941",
    "CD16 Mc / TAM": "#268f07", 
    "cDC": "#3bff8c", 
    "MSC": "#cfc10a"
}
adata_cleaned.uns['subset_colors'] = [palette[c] for c in adata_cleaned.obs['subset'].cat.categories]

fig, ax = plt.subplots(figsize=(5,5))
sc.pl.umap(adata_cleaned, color='subset', size=3, ax=ax, legend_loc='on data', show=False)
ax.set_aspect('equal')
for coll in ax.collections:
    coll.set_rasterized(True)
plt.savefig("subset_UMAP.pdf", bbox_inches="tight")


In [ ]:
adata_cleaned.layers['normalized'] = adata_cleaned.X

In [ ]:
adata_cleaned.write('annotated.h5ad')